## 1. Mã chứng khoán



1.1 Thu thập dữ liệu lịch sử - Cơ chế tải: Tự động xử lý phân trang. 

1.2. Chuẩn hóa dữ liệu: Đổi tên cột; Xử lý định dạng

1.3. Lưu trữ: lưu thành từng file CSV riêng biệt cho mỗi mã cổ phiếu (ví dụ: ACB.csv, VCB.csv). Thư mục lưu trữ là data_ohlcv.

In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime
import os

# --- 1. CẤU HÌNH CHUNG ---
STOCK_LIST = [
    'ACB', 'BID', 'CTG', 'EIB', 'HDB', 'LPB', 'MBB', 'MSB', 
    'NAB', 'OCB', 'SHB', 'SSB', 'STB', 'TCB', 'TPB', 'VCB', 
    'VIB', 'VPB'
]

# --- THAY ĐỔI 1: Lấy từ 2019 ---
START_DATE_CAFEF = '01/01/2019'
END_DATE_CAFEF = '01/01/2025' # Giữ nguyên để bao gồm hết 2024

DATA_FOLDER = 'data_ohlcv' # Đổi tên thư mục để tránh ghi đè file cũ
CAFEF_API_URL = 'https://cafef.vn/du-lieu/Ajax/PageNew/DataHistory/PriceHistory.ashx'
REQUEST_HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Referer': 'https://cafef.vn/'
}

# --- 2. HÀM TẢI DỮ LIỆU (Đã nâng cấp để lấy OHLCV-A) ---
def get_stock_data_from_cafef(symbol, start_date, end_date):
    """
    Tải dữ liệu OHLCV-A cho 1 mã cổ phiếu từ CafeF, xử lý phân trang.
    """
    print(f"    Đang tải {symbol} từ CafeF...")
    all_data = []
    page = 1
    session = requests.Session()
    
    while True:
        params = {
            'Symbol': symbol,
            'StartDate': start_date,
            'EndDate': end_date,
            'PageIndex': page,
            'PageSize': 50
        }
        
        try:
            response = session.get(CAFEF_API_URL, params=params, headers=REQUEST_HEADERS, timeout=10)
            response.raise_for_status()
            
            try:
                response_json = response.json()
            except requests.exceptions.JSONDecodeError:
                print(f"    LỖI JSON: CafeF trả về text lỗi cho {symbol} trang {page}.")
                break

            if 'Data' in response_json and 'Data' in response_json['Data']:
                actual_data_list = response_json['Data']['Data']
            else:
                print(f"    LỖI CẤU TRÚC: JSON trả về cho {symbol} không có key 'Data' bên trong.")
                break

            if not actual_data_list:
                print(f"    -> {symbol}: Trang {page} rỗng, kết thúc tải.")
                break
                
            all_data.extend(actual_data_list)
            page += 1
            time.sleep(0.2) # Giữ nguyên sleep

        except requests.RequestException as e:
            print(f"    LỖI MẠNG/HTTP khi tải {symbol} (trang {page}): {e}")
            break
            
    if not all_data:
        print(f"    -> Không tải được dữ liệu cho {symbol}")
        return pd.DataFrame()

    # --- THAY ĐỔI 2: Xử lý DataFrame để lấy OHLCV-A ---
    try:
        df = pd.DataFrame(all_data)
        
        # 2a. Các cột cần thiết từ CafeF
        COLUMNS_TO_KEEP = [
            'Ngay', 
            'GiaMoCua', 
            'GiaCaoNhat', 
            'GiaThapNhat', 
            'GiaDongCua', 
            'GiaDieuChinh', 
            'KhoiLuongKhopLenh' # Thường là 'KhoiLuongKhopLenh' hoặc 'KLGDKL'
        ]
        
        # 2b. Đổi tên sang chuẩn tiếng Anh (quan trọng cho pandas-ta)
        RENAME_MAP = {
            'Ngay': 'date_str', # Tạm thời
            'GiaMoCua': 'open',
            'GiaCaoNhat': 'high',
            'GiaThapNhat': 'low',
            'GiaDongCua': 'close',
            'GiaDieuChinh': 'adjusted_close',
            'KhoiLuongKhopLenh': 'volume'
        }
        
        # 2c. Kiểm tra xem có đủ các cột không
        if not all(col in df.columns for col in COLUMNS_TO_KEEP):
            print(f"    LỖI CỘT: Dữ liệu cho {symbol} thiếu các cột OHLCV.")
            print(f"    Các cột có: {list(df.columns)}")
            # Thử kiểm tra tên cột khối lượng thay thế
            if 'KLGDKL' in df.columns and 'KhoiLuongKhopLenh' not in df.columns:
                print("    Tìm thấy 'KLGDKL', đang thử lại...")
                COLUMNS_TO_KEEP.remove('KhoiLuongKhopLenh')
                COLUMNS_TO_KEEP.append('KLGDKL')
                RENAME_MAP['KLGDKL'] = 'volume'
            else:
                return pd.DataFrame() # Bỏ qua nếu vẫn thiếu
        
        # 2d. Chọn và đổi tên cột
        df = df[COLUMNS_TO_KEEP]
        df = df.rename(columns=RENAME_MAP)
        
        # 2e. Chuyển đổi ngày và đặt làm index
        df['date'] = pd.to_datetime(df['date_str'], format='%d/%m/%Y')
        df = df.set_index('date')
        
        # 2f. Đảm bảo đúng kiểu dữ liệu (RẤT QUAN TRỌNG)
        numeric_cols = ['open', 'high', 'low', 'close', 'adjusted_close', 'volume']
        # Xóa dấu phẩy (,) nếu có, ví dụ: 1,200
        for col in numeric_cols:
            if df[col].dtype == 'object':
                 df[col] = df[col].astype(str).str.replace(',', '', regex=False)
                     
        df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
        
        # 2g. Sắp xếp lại thứ tự cột chuẩn và trả về
        df = df[['open', 'high', 'low', 'close', 'adjusted_close', 'volume']]
        
        print(f"    -> Tải và xử lý OHLCV-A cho {symbol} thành công ({len(df)} dòng).")
        return df
        
    except KeyError as e:
        print(f"    LỖI KEY_ERROR: {e}. Cấu trúc JSON cho {symbol} không như mong đợi.")
        if 'df' in locals():
            print(f"    Các cột có: {list(df.columns)}")
        return pd.DataFrame()
    except Exception as e:
        print(f"    LỖI XỬ LÝ PANDAS cho {symbol}: {e}")
        return pd.DataFrame()


# --- 4. CHẠY CHƯƠNG TRÌNH CHÍNH ---
print("BẮT ĐẦU QUÁ TRÌNH TẢI VÀ LƯU DỮ LIỆU CỔ PHIẾU (OHLCV-A)")

os.makedirs(DATA_FOLDER, exist_ok=True)
print(f"Dữ liệu sẽ được lưu vào thư mục: '{DATA_FOLDER}/'")

print("\nBắt đầu tải dữ liệu cổ phiếu từ CafeF...")

successful_count = 0
failed_count = 0

for stock in STOCK_LIST:
    stock_df = get_stock_data_from_cafef(stock, START_DATE_CAFEF, END_DATE_CAFEF)
    
    if not stock_df.empty:
        try:
            file_path = os.path.join(DATA_FOLDER, f"{stock}.csv")
            stock_df.to_csv(file_path, index=True) # Lưu file CSV với đầy đủ cột
            
            print(f"    ✅ Đã lưu {stock} vào {file_path}")
            successful_count += 1
            
        except (OSError, PermissionError) as e:
            print(f"    ❌ LỖI KHI LƯU FILE {stock}: {e}")
            failed_count += 1
    else:
        print(f"    ❌ Bỏ qua {stock} do không có dữ liệu (lỗi đã được báo ở trên).")
        failed_count += 1

print("\n--- HOÀN TẤT ---")
print(f"Tải thành công: {successful_count} mã")
print(f"Tải thất bại:     {failed_count} mã")

## 2. Chỉ số thị trường

2.1. Tải dữ liệu lịch sử - Thư viện vnstock.

2.2. Chuẩn hóa dữ liệu: Đưa về chuẩn chung cho phân tích kỹ thuật - Đổi tên cột; Xử lý thời gian: Chuyển cột thời gian về dạng datetime và đặt làm Index (mục lục) cho DataFrame; Sắp xếp: Đảm bảo dữ liệu được sắp xếp theo thứ tự thời gian tăng dần.

2.3. Lưu trữ: Lưu vào file data_ohlcv/VNINDEX_OHLCV.csv.

In [ ]:
from vnstock import Vnstock
import pandas as pd
from datetime import datetime
import os # Thêm thư viện os

# --- 1. CẤU HÌNH ---
# Lưu vào cùng thư mục với dữ liệu CafeF
DATA_FOLDER = 'data_ohlcv' 
START_DATE_DEFAULT = '2019-01-01' # THAY ĐỔI: Lấy từ 2019
END_DATE_PROJECT = '2025-01-01'   # Để bao gồm toàn bộ năm 2024

def get_vnindex_data(start_date=START_DATE_DEFAULT, end_date=None, data_folder=DATA_FOLDER):
    """
    Lấy dữ liệu lịch sử VNINDEX (OHLCV) đã chuẩn hóa và lưu thành CSV
    
    Parameters:
    - start_date: ngày bắt đầu (mặc định: '2019-01-01')
    - end_date: ngày kết thúc (mặc định: ngày hiện tại)
    - data_folder: thư mục lưu file
    """
    
    # Nếu không có end_date, lấy ngày hiện tại
    if end_date is None:
        end_date = datetime.now().strftime('%Y-%m-%d')
    
    print(f"Đang tải dữ liệu VNINDEX từ {start_date} đến {end_date}...")
    
    # Đảm bảo thư mục tồn tại
    os.makedirs(data_folder, exist_ok=True)
    
    df = None
    sources_to_try = ['VCI', 'TCBS'] # Các nguồn để thử
    
    # Tự động thử các nguồn
    for source in sources_to_try:
        try:
            print(f"    -> Đang thử nguồn: {source}")
            stock = Vnstock().stock(symbol='VNINDEX', source=source)
            df = stock.quote.history(start=start_date, end=end_date)
            
            if df is not None and not df.empty:
                print(f"    -> Tải thành công từ {source}.")
                break # Tải thành công, thoát vòng lặp
            else:
                print(f"    -> Nguồn {source} trả về rỗng.")
        
        except Exception as e:
            print(f"    ✗ Lỗi với nguồn {source}: {e}")

    # Kiểm tra lần cuối sau khi thử hết các nguồn
    if df is None or df.empty:
        print("✗ Lỗi: Không thể tải dữ liệu VNINDEX từ mọi nguồn.")
        return None
        
    # --- THAY ĐỔI 2: XỬ LÝ VÀ CHUẨN HÓA DỮ LIỆU ---
    try:
        # 1. Đặt Index (nếu chưa phải là index)
        if 'time' in df.columns:
            df['date'] = pd.to_datetime(df['time'])
            df.set_index('date', inplace=True)
        elif 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'])
            df.set_index('date', inplace=True)
        
        # 2. Đổi tên cột sang chữ thường (RẤT QUAN TRỌNG cho pandas-ta)
        df.columns = df.columns.str.lower()
        
        # 3. Chọn các cột OHLCV chuẩn
        # VNINDEX không có 'adjusted_close', chúng ta dùng 'close'
        required_cols = ['open', 'high', 'low', 'close', 'volume']
        
        if all(col in df.columns for col in required_cols):
            df = df[required_cols]
        else:
            print("✗ Lỗi: Dữ liệu trả về thiếu các cột OHLCV.")
            print(f"    Các cột có: {list(df.columns)}")
            return None
            
        # 4. Sắp xếp lại theo ngày
        df.sort_index(ascending=True, inplace=True)
            
        # 5. Lưu file
        filename = f'VNINDEX_OHLCV.csv' # Đặt tên file đơn giản
        file_path = os.path.join(data_folder, filename)
        df.to_csv(file_path, encoding='utf-8-sig')
        
        print(f"✓ Đã lưu dữ liệu chuẩn hóa vào file: {file_path}")
        print(f"✓ Tổng số dòng dữ liệu: {len(df)}")
        print(f"\nXem trước dữ liệu (đã chuẩn hóa OHLCV):")
        print(df.head())
        
        return df
        
    except Exception as e:
        print(f"✗ Lỗi khi xử lý DataFrame: {e}")
        return None

# Sử dụng hàm
if __name__ == "__main__":
    # Lấy dữ liệu từ 2019 đến hết 2024 (theo mốc 01/01/2025)
    vnindex_data = get_vnindex_data(start_date=START_DATE_DEFAULT, end_date=END_DATE_PROJECT)

## 3. Tổng hợp và chuẩn hóa dữ liệu

3.1. Đồng bộ hóa dữ liệu: Mã nguồn đọc file chỉ số thị trường VNINDEX_OHLCV.csv và tất cả các file cổ phiếu trong thư mục data_ohlcv, sau đó chuẩn hóa tên cột, gán thêm cột ticker cho từng bảng dữ liệu và đảm bảo mỗi file chỉ giữ lại các cột chuẩn. Tất cả dữ liệu được xử lý theo cùng một định dạng Long Format và được đưa vào một danh sách để chuẩn bị gộp.

3.2. Sàng lọc cổ phiếu: Trong quá trình duyệt danh sách cổ phiếu, mã nguồn tự động bỏ qua các cổ phiếu nằm trong REMOVE_STOCKS, vì đây là các mã niêm yết muộn hoặc thiếu dữ liệu dài hạn.

3.3. Làm sạch dữ liệu 

3.4. Xuất dữ liệu :Lưu kết quả cuối cùng vào file all_data_merged.csv.

In [ ]:
import pandas as pd
import os
import sys

# --- 1. CẤU HÌNH ---
DATA_FOLDER = 'data_ohlcv'
OUTPUT_FILE = 'all_data_merged.csv' # File kết quả dạng Long

# Danh sách mã cần xử lý
STOCK_LIST = [
    'ACB', 'BID', 'CTG', 'EIB', 'HDB', 'LPB', 'MBB',
    'MSB', 'NAB', 'OCB', 'SHB', 'SSB', 'STB', 'TCB',
    'TPB', 'VCB', 'VIB', 'VPB'
]

# Các mã muốn loại bỏ khỏi kết quả cuối cùng
REMOVE_STOCKS = ['MSB', 'NAB', 'OCB', 'SSB', 'SHB']

# Tên file và cấu hình cho VNINDEX
INDEX_FILENAME_PREFIX = 'VNINDEX_OHLCV'
INDEX_TICKER_NAME = 'VNINDEX' # Đặt tên ticker cho chỉ số này trong cột 'ticker'

# Danh sách các cột chuẩn muốn giữ lại
REQUIRED_COLUMNS = ['date', 'ticker', 'open', 'high', 'low', 'close', 'volume']

all_dataframes = []

print(f"🚀 BẮT ĐẦU QUÁ TRÌNH GỘP DỮ LIỆU (LONG FORMAT) TỪ '{DATA_FOLDER}'...\n")

# ==============================================================================
# HÀM HỖ TRỢ ĐỌC VÀ CHUẨN HÓA
# ==============================================================================
def process_file(file_path, ticker_name, is_index=False):
    try:
        # 1. Đọc file
        if not os.path.exists(file_path):
            print(f"   ⚠️ Không tìm thấy file: {file_path}. Bỏ qua.")
            return None
            
        df = pd.read_csv(file_path, parse_dates=['date'])
        
        # 2. Thêm cột 'ticker' (Quan trọng nhất của Long Format)
        df['ticker'] = ticker_name
        
        # 3. Chuẩn hóa tên cột về chữ thường
        df.columns = df.columns.str.lower()
        
        # 4. Xử lý cột 'adjusted_close' (VNINDEX thường không có, Cổ phiếu thì có)
        # Nếu bạn muốn giữ adjusted_close, thêm vào danh sách cột
        cols_to_take = REQUIRED_COLUMNS.copy()
        if 'adjusted_close' in df.columns:
             cols_to_take.append('adjusted_close')
        elif is_index:
             # Với VNINDEX, adjusted_close chính là close
             df['adjusted_close'] = df['close']
             cols_to_take.append('adjusted_close')

        # 5. Chỉ lấy các cột cần thiết để đảm bảo đồng nhất
        # (Sử dụng intersection để tránh lỗi nếu thiếu cột nào đó lạ)
        available_cols = [c for c in cols_to_take if c in df.columns]
        df = df[available_cols]
        
        return df
        
    except Exception as e:
        print(f"   ❌ Lỗi khi xử lý {ticker_name}: {e}")
        return None

# ==============================================================================
# 2. XỬ LÝ FILE CHỈ SỐ (VNINDEX)
# ==============================================================================
index_file_path = os.path.join(DATA_FOLDER, f"{INDEX_FILENAME_PREFIX}.csv")
print(f"📄 Đang xử lý chỉ số thị trường ({INDEX_TICKER_NAME})...")

index_df = process_file(index_file_path, INDEX_TICKER_NAME, is_index=True)
if index_df is not None:
    all_dataframes.append(index_df)
    print(f"   -> Thêm thành công {len(index_df)} dòng dữ liệu VNINDEX.")
else:
    print("   ❌ Lỗi nghiêm trọng: Không có dữ liệu VNINDEX.")
    sys.exit()

# ==============================================================================
# 3. XỬ LÝ CÁC FILE CỔ PHIẾU
# ==============================================================================
print(f"\n📄 Đang xử lý {len(STOCK_LIST)} mã cổ phiếu...")
count = 0

for stock in STOCK_LIST:
    # Bỏ qua nếu mã nằm trong danh sách loại bỏ (xử lý sớm để tiết kiệm bộ nhớ)
    if stock in REMOVE_STOCKS:
        print(f"   -> Bỏ qua {stock} (nằm trong danh sách loại bỏ).")
        continue
        
    file_path = os.path.join(DATA_FOLDER, f"{stock}.csv")
    stock_df = process_file(file_path, stock)
    
    if stock_df is not None:
        all_dataframes.append(stock_df)
        count += 1

print(f"   -> Đã thêm thành công {count} mã cổ phiếu.")

# ==============================================================================
# 4. GỘP VÀ LƯU DỮ LIỆU
# ==============================================================================
if not all_dataframes:
    print("\n❌ Không có dữ liệu nào để gộp.")
    sys.exit()

print("\n🔄 Đang gộp tất cả dữ liệu (Stacking)...")

# Dùng axis=0 để xếp chồng các dataframe lên nhau (Long Format)
final_df = pd.concat(all_dataframes, axis=0, ignore_index=True)

# Sắp xếp lại theo Ticker -> Date
print("✨ Đang sắp xếp dữ liệu...")
final_df = final_df.sort_values(by=['ticker', 'date'])

# Lưu file
output_path = os.path.join(DATA_FOLDER, OUTPUT_FILE)
try:
    final_df.to_csv(output_path, index=False)
    print(f"\n✅ HOÀN TẤT! Đã lưu file Long Format đầy đủ cột tại: {output_path}")
    
    print("\n--- MẪU DỮ LIỆU (5 dòng đầu) ---")
    print(final_df.head())
    
    print("\n--- THỐNG KÊ ---")
    print(f"Tổng số dòng: {len(final_df)}")
    print(f"Các cột giữ lại: {list(final_df.columns)}")
    print(f"Số lượng mã: {final_df['ticker'].nunique()}")
    print(f"Danh sách mã: {final_df['ticker'].unique()}")

except Exception as e:
    print(f"\n❌ LỖI KHI LƯU FILE: {e}")

## 4. Kiểm tra dữ liệu

4.1. Kiểm tra tính toàn vẹn của file:
- Xác nhận xem file data_final.csv có tồn tại và có mở được không.
- Xem nhanh 5 dòng đầu để đảm bảo cấu trúc bảng đúng ý muốn.
- Kiểm tra định dạng: Đảm bảo cột date đã được chuyển đúng về dạng thời gian (datetime)

4.2. Kiểm tra dữ liệu thiếu: Có ô nào bị NaN (Not a Number) hay Null không. 

4.3. Kiểm tra logic loại bỏ: Xác minh lại xem các mã cổ phiếu muốn loại bỏ (MSB, NAB, OCB, SSB, SHB) có thực sự đã biến mất khỏi dữ liệu chưa. 

4.4.Kiểm tra trùng lặp: Có cặp (date, ticker) nào bị lặp lại 2 lần không. 

4.5. Kiểm tra phạm vi thời gian: Xác nhận dữ liệu có bắt đầu từ năm 2019 (để đảm bảo có giai đoạn "warm-up" cho các chỉ báo kỹ thuật như MA200 hoạt động) và kết thúc đúng hạn (không vượt quá 01/01/2025) hay không.

In [ ]:
import pandas as pd
import os
import sys

# ==============================================================================
# PHẦN 1: CẤU HÌNH CHUNG
# ==============================================================================
DATA_FOLDER = 'data_ohlcv'
INPUT_FILE = 'all_data_merged.csv'  # File này bây giờ ĐÃ LÀ dạng Long Format (chứa OHLCV)
OUTPUT_FILE = 'data_final.csv'      # File đầu ra (đã được làm sạch và kiểm tra)

input_path = os.path.join(DATA_FOLDER, INPUT_FILE)
output_path = os.path.join(DATA_FOLDER, OUTPUT_FILE)

# Danh sách các mã cần loại bỏ (Check lại lần cuối cho chắc)
removed_stocks_check = ['MSB', 'NAB', 'OCB', 'SSB', 'SHB']

# ==============================================================================
# PHẦN 2: CHUẨN BỊ DỮ LIỆU (KHÔNG CẦN MELT NỮA)
# ==============================================================================
print("="*60)
print("BƯỚC 1: CHUẨN BỊ VÀ SÀNG LỌC DỮ LIỆU (LONG FORMAT)")
print("="*60)

try:
    # 1. Đọc dữ liệu
    if not os.path.exists(input_path):
        print(f"❌ LỖI: Không tìm thấy file đầu vào: {input_path}")
        sys.exit()
        
    # Đọc file, pandas sẽ tự nhận diện các cột OHLCV
    df = pd.read_csv(input_path, parse_dates=['date'])
    
    print(f"✓ Đã đọc file: {INPUT_FILE}")
    print(f"  - Số dòng: {len(df)}")
    print(f"  - Các cột hiện có: {list(df.columns)}")

    # 2. Kiểm tra cấu trúc Long Format
    if 'ticker' not in df.columns:
        print("❌ LỖI: File đầu vào không có cột 'ticker'. Vui lòng kiểm tra lại bước gộp dữ liệu.")
        sys.exit()

    # 3. Lọc bỏ các mã rác (Lớp bảo vệ thứ 2)
    # Mặc dù bước trước đã lọc, nhưng làm lại ở đây để đảm bảo data_final sạch tuyệt đối
    print(f"  - Đang rà soát loại bỏ các mã: {removed_stocks_check}")
    df = df[~df['ticker'].isin(removed_stocks_check)]

    # 4. Sắp xếp lại (Theo Mã -> Ngày)
    print("⏳ Đang sắp xếp dữ liệu...")
    df = df.sort_values(by=['ticker', 'date'])

    # 5. Lưu file kết quả (Đây là file chính thức để dùng về sau)
    df.to_csv(output_path, index=False)
    print(f"✅ THÀNH CÔNG! Đã lưu file dữ liệu chuẩn tại: {output_path}")
    
except Exception as e:
    print(f"❌ LỖI NGHIÊM TRỌNG KHI XỬ LÝ: {e}")
    sys.exit()

# ==============================================================================
# PHẦN 3: KIỂM TRA DỮ LIỆU (VALIDATION)
# ==============================================================================
print("\n" + "="*60)
print(f"BƯỚC 2: KIỂM TRA CHẤT LƯỢNG FILE: {OUTPUT_FILE}")
print("="*60)

# 1. Tải lại file vừa tạo để kiểm tra khách quan
try:
    df_check = pd.read_csv(output_path)
    print(f"✓ Đã tải lại file để kiểm tra: {OUTPUT_FILE}")
except Exception as e:
    print(f"❌ LỖI KHI ĐỌC FILE ĐẦU RA: {e}")
    sys.exit()

print("\n--- 1. Xem trước dữ liệu (5 dòng đầu) ---")
# In ra để bạn thấy rõ các cột open, high, low, close, volume vẫn còn nguyên
print(df_check.head())

print("\n--- 2. Kiểm tra Kiểu dữ liệu ---")
try:
    df_check['date'] = pd.to_datetime(df_check['date'])
    print("✓ Cột 'date' chuẩn định dạng datetime.")
except Exception as e:
    print(f"❌ Lỗi định dạng cột 'date': {e}")

print("\n--- 3. Kiểm tra Giá trị thiếu (NaN) ---")
missing_report = df_check.isnull().sum()
if missing_report.sum() == 0:
    print("✓ HOÀN HẢO: Không có giá trị thiếu (NaN).")
else:
    print("❌ CẢNH BÁO: Có giá trị thiếu:")
    print(missing_report[missing_report > 0])
    # Tự động sửa lỗi: Xóa dòng thiếu
    print("  -> Đang tự động xóa các dòng NaN...")
    df_check = df_check.dropna()
    df_check.to_csv(output_path, index=False)
    print("  -> Đã xóa xong và cập nhật file.")

print("\n--- 4. Kiểm tra Cột 'ticker' ---")
tickers_list = df_check['ticker'].unique()
print(f"✓ Tổng số mã: {len(tickers_list)}")
# Kiểm tra mã bị loại
found_removed = [t for t in tickers_list if t in removed_stocks_check]
if not found_removed:
    print(f"✓ Tốt: Các mã rác ({', '.join(removed_stocks_check)}) không tồn tại.")
else:
    print(f"❌ CẢNH BÁO: Vẫn còn các mã cần loại bỏ: {found_removed}")

print("\n--- 5. Kiểm tra Trùng lặp ---")
duplicates = df_check.duplicated(subset=['date', 'ticker']).sum()
if duplicates == 0:
    print("✓ HOÀN HẢO: Không có dữ liệu trùng lặp.")
else:
    print(f"❌ CẢNH BÁO: Có {duplicates} dòng bị trùng lặp.")

print("\n--- 6. Kiểm tra Phạm vi thời gian ---")
if 'date' in df_check.columns:
    min_date = df_check['date'].min().date()
    max_date = df_check['date'].max().date()
    print(f"✓ Dữ liệu từ: {min_date} đến {max_date}")
    
    if min_date.year > 2019:
         print("⚠️ Lưu ý: Dữ liệu bắt đầu sau năm 2019.")
    else:
         print("✓ Tốt: Dữ liệu bao gồm giai đoạn 2019.")

print("\n" + "="*60)
print("HOÀN TẤT QUÁ TRÌNH.")
print("="*60)

## 5. Tỷ suất sinh lợi Logarit theo ngày cho từng mã cổ phiếu

5.1. Chuẩn bị dữ liệu: Tải file data_final.csv.
Sắp xếp lại (Sorting): Đảm bảo dữ liệu được xếp theo từng mã cổ phiếu riêng biệt, và trong mỗi mã, thời gian chạy từ quá khứ đến hiện tại.  

5.2. Tính toán cốt lõi: Mã sử dụng công thức Log Return = ln(Pt/(Pt-1)). Cơ chế groupby('ticker') đóng vai trò "cách ly". Nó đảm bảo việc lấy giá ngày hôm trước (shift(1)) chỉ diễn ra trong nội bộ từng mã cổ phiếu.

5.3. Cập nhật file: Sau khi tính toán xong, một cột mới tên là log_return được thêm vào DataFrame; lưu đè kết quả (bao gồm cột mới) vào chính file data_final.csv.   

In [ ]:
import pandas as pd
import numpy as np
import os
import sys

# ==============================================================================
# 1. CẤU HÌNH
# ==============================================================================
DATA_FOLDER = 'data_ohlcv'
FILENAME = "data_final.csv"

# Đường dẫn đầy đủ: data_ohlcv/data_final.csv
input_file_path = os.path.join(DATA_FOLDER, FILENAME)

print(f"🚀 BẮT ĐẦU TÍNH TOÁN TỶ SUẤT SINH LỢI (LOG RETURNS) TRÊN FILE: {input_file_path}")

# ==============================================================================
# 2. TẢI DỮ LIỆU
# ==============================================================================
try:
    if not os.path.exists(input_file_path):
        print(f"❌ LỖI: Không tìm thấy tệp '{input_file_path}'.")
        sys.exit()

    # Quan trọng: parse_dates=['date'] để pandas hiểu đúng cột ngày tháng
    df = pd.read_csv(input_file_path, parse_dates=['date'])
    print(f"✓ Đã tải thành công tệp. Tổng số dòng: {len(df)}")
    
except Exception as e:
    print(f"❌ Lỗi khi tải tệp: {e}")
    sys.exit()

# ==============================================================================
# 3. TÍNH TOÁN LOG RETURNS (DÙNG ADJUSTED CLOSE)
# ==============================================================================
if df is not None:
    
    # Kiểm tra xem cột adjusted_close có tồn tại không
    if 'adjusted_close' not in df.columns:
        print("❌ LỖI NGHIÊM TRỌNG: Không tìm thấy cột 'adjusted_close' trong dữ liệu.")
        print("   Vui lòng kiểm tra lại bước gộp dữ liệu (cần đảm bảo giữ lại cột này).")
        sys.exit()

    # A. Sắp xếp dữ liệu (Rất quan trọng)
    print("\n⏳ Đang sắp xếp lại dữ liệu (Ticker -> Date)...")
    df = df.sort_values(by=['ticker', 'date'])

    # B. Tính toán
    print("⏳ Đang tính toán Log Returns dựa trên GIÁ ĐIỀU CHỈNH (Adjusted Close)...")
    
    # Sử dụng groupby để đảm bảo việc lấy giá hôm trước (shift) chỉ diễn ra trong nội bộ từng mã
    # Công thức MỚI: ln(Adj_Close_t / Adj_Close_t-1)
    df['prev_adj_close'] = df.groupby('ticker')['adjusted_close'].shift(1)
    
    # Tính log return
    df['log_return'] = np.log(df['adjusted_close'] / df['prev_adj_close'])

    # Xóa cột phụ 'prev_adj_close' để file gọn gàng
    df.drop(columns=['prev_adj_close'], inplace=True)

    # C. Kiểm tra kết quả
    print("\n--- Xem trước kết quả (5 dòng đầu có dữ liệu) ---")
    # Hiện cả close và adjusted_close để so sánh
    cols_to_show = ['date', 'ticker', 'close', 'adjusted_close', 'log_return']
    
    # Lấy 5 dòng ngẫu nhiên KHÔNG phải NaN để kiểm tra
    print(df[df['log_return'].notna()][cols_to_show].head())

    # ==============================================================================
    # 4. LƯU KẾT QUẢ
    # ==============================================================================
    try:
        # Lưu đè lên file cũ
        df.to_csv(input_file_path, index=False)
        print(f"\n✅ THÀNH CÔNG! Đã cập nhật cột 'log_return' (chuẩn adjusted) vào file: {input_file_path}")
        
        # Thống kê nhanh
        nan_count = df['log_return'].isnull().sum()
        ticker_count = df['ticker'].nunique()
        print(f"   - Tổng số mã cổ phiếu: {ticker_count}")
        print(f"   - Số lượng giá trị NaN: {nan_count} (Bằng số lượng mã là đúng, do dòng đầu tiên mỗi mã là NaN)")
        
    except Exception as e:
        print(f"❌ Lỗi khi lưu tệp: {e}")

## 6. Tạo đặc trưng

6.1. Tính toán Chỉ báo Kỹ thuật:Sử dụng thư viện ta để tính hàng loạt chỉ báo cho từng mã cổ phiếu riêng biệt.
- Xu hướng (Trend): SMA10, SMA50 (đường trung bình động), MACD (phân kỳ hội tụ).
- Động lượng (Momentum): RSI (sức mạnh tương đối), Stochastic Oscillator.
- Biến động (Volatility): ATR (biên độ dao động trung bình), Bollinger Bands, Độ biến động lịch sử (Historical Volatility).
- Dòng tiền (Volume): OBV (On-Balance Volume), CMF (Chaikin Money Flow)

6.2. Tính toán Chỉ báo Thị trường : Tách riêng dữ liệu VNINDEX để tính các chỉ báo vĩ mô. Mã tính toán RSI, ATR, Volatility riêng cho VNINDEX và đặt tiền tố VNI_ (ví dụ: VNI_RSI_14).

6.3. Hợp nhất dữ liệu: Dùng pd.merge để "gán" các chỉ số của VNINDEX vào từng dòng dữ liệu của từng cổ phiếu. Ví dụ: Dòng dữ liệu ngày 18/11/2025 của mã VCB sẽ có thêm các cột như VNI_RSI_14 (RSI của thị trường ngày hôm đó).

6.4. Chuẩn bị cho Machine Learning: loại bỏ dòng dữ liệu riêng của VNINDEX khỏi file cuối cùng. File kết quả chỉ còn lại các mã cổ phiếu, nhưng mỗi mã đã được thêm các cột dữ liệu thông tin về thị trường. Kết quả: File data_final.csv lúc này đã trở thành một bộ dữ liệu huấn luyện (Training Dataset) hoàn chỉnh, giàu thông tin, sẵn sàng để đưa vào các mô hình dự báo.

In [ ]:
# Các chỉ báo kĩ thuật
import pandas as pd
import numpy as np
import ta # Thư viện Technical Analysis (cài đặt bằng: pip install ta)
import warnings
import os
import sys

# Tắt các cảnh báo từ thư viện 'ta' và 'pandas'
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# --- 1. CẤU HÌNH ---
DATA_FOLDER = 'data_ohlcv'
FILENAME = "data_final.csv"

# Đường dẫn đầy đủ
FILE_PATH = os.path.join(DATA_FOLDER, FILENAME)

# --- CÁC HÀM TÍNH TOÁN ---

def calculate_stock_indicators(group):
    """
    Hàm này tính toán tất cả các chỉ báo cho một nhóm (một mã ticker).
    """
    # Đảm bảo dữ liệu được sắp xếp theo ngày
    group = group.sort_values('date')
    
    # 1. Chỉ báo Xu hướng (Trend)
    group['SMA_10'] = ta.trend.SMAIndicator(group['close'], window=10).sma_indicator()
    group['SMA_50'] = ta.trend.SMAIndicator(group['close'], window=50).sma_indicator()
    
    macd = ta.trend.MACD(group['close'], window_slow=26, window_fast=12, window_sign=9)
    group['MACD'] = macd.macd()
    group['MACD_signal'] = macd.macd_signal()
    group['MACD_hist'] = macd.macd_diff()
    
    # 2. Chỉ báo Động lượng (Momentum)
    group['RSI_14'] = ta.momentum.RSIIndicator(group['close'], window=14).rsi()
    group['Stoch_K_14'] = ta.momentum.StochasticOscillator(
        group['high'], group['low'], group['close'], window=14
    ).stoch()
    
    # 3. Chỉ báo Biến động (Volatility)
    group['ATR_14'] = ta.volatility.AverageTrueRange(
        group['high'], group['low'], group['close'], window=14
    ).average_true_range()
    
    # 'log_return' đã có sẵn từ bước trước, dùng nó để tính biến động lịch sử
    if 'log_return' in group.columns:
        group['Historical_Vol_20'] = group['log_return'].rolling(window=20).std()
    else:
        # Phòng trường hợp chưa tính log_return, tính tạm thời
        group['log_return_temp'] = np.log(group['close'] / group['close'].shift(1))
        group['Historical_Vol_20'] = group['log_return_temp'].rolling(window=20).std()
        group.drop(columns=['log_return_temp'], inplace=True)
    
    bb = ta.volatility.BollingerBands(group['close'], window=20, window_dev=2)
    group['BB_Middle'] = bb.bollinger_mavg()
    group['BB_Upper'] = bb.bollinger_hband()
    group['BB_Lower'] = bb.bollinger_lband()
    # Tính BB_Width chuẩn hóa
    group['BB_Width'] = (group['BB_Upper'] - group['BB_Lower']) / group['BB_Middle']
    
    # 4. Chỉ báo Khối lượng (Volume)
    group['OBV'] = ta.volume.OnBalanceVolumeIndicator(group['close'], group['volume']).on_balance_volume()
    group['CMF_20'] = ta.volume.ChaikinMoneyFlowIndicator(
        group['high'], group['low'], group['close'], group['volume'], window=20
    ).chaikin_money_flow()
    
    return group

def calculate_market_indicators(market_df):
    """
    Hàm này tính toán các chỉ báo cho VNINDEX và thêm tiền tố VNI_.
    """
    market_df = market_df.sort_values('date')
    
    # Kiểm tra xem log_return có sẵn không
    if 'log_return' in market_df.columns:
        market_df['VNI_log_return'] = market_df['log_return']
        market_df['VNI_Hist_Vol_20'] = market_df['log_return'].rolling(window=20).std()
    else:
        # Tính lại nếu thiếu
        market_df['VNI_log_return'] = np.log(market_df['close'] / market_df['close'].shift(1))
        market_df['VNI_Hist_Vol_20'] = market_df['VNI_log_return'].rolling(window=20).std()

    market_df['VNI_ATR_14'] = ta.volatility.AverageTrueRange(
        market_df['high'], market_df['low'], market_df['close'], window=14
    ).average_true_range()
    
    market_df['VNI_RSI_14'] = ta.momentum.RSIIndicator(
        market_df['close'], window=14
    ).rsi()
    
    # Chọn các cột để hợp nhất
    market_features = [
        'date', 
        'VNI_log_return', 
        'VNI_Hist_Vol_20', 
        'VNI_ATR_14', 
        'VNI_RSI_14'
    ]
    return market_df[market_features].copy()

# --- CHƯƠNG TRÌNH CHÍNH ---

print(f"🚀 BẮT ĐẦU TÍNH TOÁN CHỈ BÁO KỸ THUẬT TRÊN FILE: {FILE_PATH}")

# 1. Tải dữ liệu
try:
    if not os.path.exists(FILE_PATH):
        print(f"❌ LỖI: Không tìm thấy tệp '{FILE_PATH}'.")
        sys.exit()
        
    df = pd.read_csv(FILE_PATH, parse_dates=['date'])
    print(f"✓ Đã tải dữ liệu thành công. Tổng số dòng: {len(df)}")
except Exception as e:
    print(f"❌ Lỗi khi đọc file: {e}")
    sys.exit()

# Sắp xếp dữ liệu
df = df.sort_values(by=['ticker', 'date'])

# --- Bước 2: Tách và Tính toán (Thị trường - VNINDEX) ---
print("\n--- 1. Đang tính toán chỉ báo cho thị trường (VNINDEX)... ---")

# Kiểm tra xem VNINDEX có trong dữ liệu không
if 'VNINDEX' not in df['ticker'].unique():
    print("❌ LỖI: Không tìm thấy mã 'VNINDEX' trong cột 'ticker'.")
    print("   Vui lòng kiểm tra lại bước gộp dữ liệu (merge_long_format).")
    sys.exit()

df_market = df[df['ticker'] == 'VNINDEX'].copy()
df_market_features = calculate_market_indicators(df_market)
print("✓ Đã tính xong chỉ báo thị trường.")

# --- Bước 3: Tách và Tính toán (Cổ phiếu) ---
print("\n--- 2. Đang tính toán chỉ báo cho các cổ phiếu riêng lẻ... ---")
df_stocks = df[df['ticker'] != 'VNINDEX'].copy()

unique_tickers = df_stocks['ticker'].nunique()
print(f"   Đang áp dụng tính toán cho {unique_tickers} mã cổ phiếu...")

# Sử dụng groupby.apply() để tính toán chỉ báo cho từng mã cổ phiếu
# include_groups=False để tránh warning trong pandas bản mới (nếu có)
try:
    df_stocks_features = df_stocks.groupby('ticker', group_keys=False).apply(calculate_stock_indicators)
except TypeError:
    # Fallback cho phiên bản pandas cũ hơn
    df_stocks_features = df_stocks.groupby('ticker').apply(calculate_stock_indicators)
    df_stocks_features = df_stocks_features.reset_index(drop=True)

print("✓ Đã tính xong chỉ báo cho tất cả cổ phiếu.")

# --- Bước 4: Hợp nhất (Merge) ---
print("\n--- 3. Đang hợp nhất dữ liệu thị trường vào dữ liệu cổ phiếu... ---")

# Lưu ý: df_stocks_features bây giờ chứa các chỉ báo của cổ phiếu
# Chúng ta merge df_market_features (chỉ chứa date và chỉ số VNI) vào đó
df_final = pd.merge(
    df_stocks_features, 
    df_market_features, 
    on='date',            # Khóa chung để hợp nhất
    how='left'            # Giữ tất cả các hàng cổ phiếu
)
print("✓ Đã hợp nhất xong.")

# --- Bước 5: Lưu kết quả ---
print(f"\n--- 4. Đang lưu bộ dữ liệu hoàn chỉnh vào {FILE_PATH}... ---")
try:
    # Sắp xếp lại lần cuối cho đẹp
    df_final = df_final.sort_values(by=['ticker', 'date'])
    
    # Lưu đè file gốc
    df_final.to_csv(FILE_PATH, index=False)
    
    print("\n✅ ================== THÀNH CÔNG! ==================")
    print(f"Đã lưu dữ liệu ML-ready vào: {FILE_PATH}")
    print(f"File này hiện CHỈ chứa dữ liệu cổ phiếu (VNINDEX đã được tách ra và gộp vào cột VNI_...)")
    
    print("\n--- 5 dòng dữ liệu cuối cùng trong file mới ---")
    cols_preview = ['date', 'ticker', 'close', 'RSI_14', 'VNI_RSI_14'] 
    # Chỉ hiện một vài cột quan trọng để check
    print(df_final[cols_preview].tail())
    
    print("\n--- Thống kê ---")
    print(f"Số lượng cột: {len(df_final.columns)}")
    print(f"Các cột mới ví dụ: {list(df_final.columns)[-5:]}")

except Exception as e:
    print(f"\n❌ LỖI NGHIÊM TRỌNG khi lưu file: {e}")

## 7. Kiểm tra dữ liệu lần cuối

7.1. Tự động sửa lỗi dữ liệu khuyết: Xóa bỏ dữ liệu năm 2019 (dữ liệu warm up được dùng để tính toán chỉ báo cho năm 2020), đảm bảo đã xóa 4 mã thiếu dữ liệu và mã thị trường (VNINDEX) đã biến mất. Đảm bảo thời gian của dữ liệu từ 2020-2024

7.2. Cập nhật dữ liệu: Sau khi đã lọc bỏ hết NaN và kiểm tra kỹ càng, script thực hiện lưu đè lên file data_final.csv. 

In [ ]:
# Kiểm tra và Làm sạch cuối cùng (Final Validation)
import pandas as pd
import sys
import os

# --- 1. CẤU HÌNH ---
DATA_FOLDER = 'data_ohlcv'
FILE_NAME = "data_final.csv"
FILE_PATH = os.path.join(DATA_FOLDER, FILE_NAME)

# Cấu hình kiểm tra
removed_stocks_check = ['MSB', 'NAB', 'OCB', 'SSB', 'SHB', 'VNINDEX'] 
# Lưu ý: VNINDEX cũng cần được loại bỏ khỏi cột ticker vì nó đã được tách thành cột riêng (VNI_...)

expected_start_date = pd.to_datetime('2020-01-01') # Ngày bắt đầu mong muốn cho Mô hình
expected_end_date = pd.to_datetime('2025-01-01')

print(f"--- BẮT ĐẦU KIỂM TRA VÀ SỬA LỖI FILE: {FILE_PATH} ---")

# --- 2. TẢI VÀ KIỂM TRA CƠ BẢN ---
try:
    if not os.path.exists(FILE_PATH):
        print(f"❌ LỖI: Không tìm thấy file '{FILE_PATH}'.")
        sys.exit()

    df = pd.read_csv(FILE_PATH)
    print(f"✓ Đã tải thành công file. Tổng số dòng ban đầu: {len(df)}")
except Exception as e:
    print(f"❌ LỖI KHI ĐỌC FILE: {e}")
    sys.exit()

print("\n--- 1. Kiểm tra Kiểu dữ liệu (dtypes) ---")
try:
    df['date'] = pd.to_datetime(df['date'])
    print("✓ Cột 'date' đã được chuyển đổi sang datetime.")
except Exception as e:
    print(f"❌ Lỗi khi chuyển đổi 'date': {e}")
    sys.exit()

# --- 3. KIỂM TRA VÀ XỬ LÝ CÁC GIÁ TRỊ THIẾU (NaN/Null) ---
print("\n--- 2. Xử lý giá trị thiếu (NaN) do tạo chỉ báo ---")
missing_report = df.isnull().sum()
missing_total = missing_report.sum()

if missing_total > 0:
    print(f"⚠️ Phát hiện {missing_total} ô trống (NaN).")
    print("   (Điều này là BÌNH THƯỜNG do các chỉ báo như SMA50, RSI cần dữ liệu quá khứ để tính)")
    
    # HÀNH ĐỘNG SỬA LỖI: Xóa hàng NaN
    print("   -> Đang xóa các hàng chứa NaN (cắt bỏ giai đoạn warm-up)...")
    original_count = len(df)
    df = df.dropna()
    new_count = len(df)
    print(f"   ✓ Đã xóa {original_count - new_count} dòng.")
    print(f"   ✓ Số dòng còn lại: {new_count}")
else:
    print("✓ Dữ liệu đã sạch, không có NaN.")

# --- 4. LỌC THEO THỜI GIAN (QUAN TRỌNG) ---
print("\n--- 3. Lọc dữ liệu theo khung thời gian mong muốn ---")
print(f"   Yêu cầu: Từ {expected_start_date.date()} đến {expected_end_date.date()}")

# Lọc bỏ dữ liệu trước 2020 (Năm 2019 chỉ dùng để tính chỉ báo, giờ không cần nữa)
df = df[(df['date'] >= expected_start_date) & (df['date'] <= expected_end_date)]
print(f"   ✓ Đã cắt dữ liệu. Số dòng sau khi lọc ngày: {len(df)}")

if len(df) == 0:
    print("❌ LỖI: Không còn dữ liệu nào sau khi lọc ngày! Vui lòng kiểm tra lại range ngày.")
    sys.exit()

# --- 5. KIỂM TRA LOGIC ---
print("\n--- 4. Kiểm tra Logic dữ liệu ---")

# Kiểm tra mã rác
tickers_list = df['ticker'].unique()
found_removed = [t for t in tickers_list if t in removed_stocks_check]

if not found_removed:
    print(f"✓ Tốt: Các mã rác ({', '.join(removed_stocks_check)}) đã sạch bóng.")
else:
    print(f"❌ CẢNH BÁO: Vẫn tìm thấy mã cần loại bỏ: {found_removed}")
    # Tự động xóa luôn nếu còn sót
    df = df[~df['ticker'].isin(found_removed)]
    print(f"   -> Đã tự động xóa các mã này.")

# Kiểm tra trùng lặp
duplicates = df.duplicated(subset=['date', 'ticker']).sum()
if duplicates == 0:
    print(f"✓ Tốt: Không có dữ liệu trùng lặp (Ticker + Date).")
else:
    print(f"❌ CẢNH BÁO: Có {duplicates} dòng trùng lặp. Đang xử lý...")
    df = df.drop_duplicates(subset=['date', 'ticker'])

# --- 6. LƯU KẾT QUẢ ---
print("\n--- 5. LƯU FILE FINAL ---")
try:
    df.to_csv(FILE_PATH, index=False)
    print(f"✅ HOÀN TẤT! Đã ghi đè dữ liệu sạch vào: {FILE_PATH}")
    
    print("\n--- THỐNG KÊ CUỐI CÙNG ---")
    print(df.head())
    print("--------------------------")
    print(df.info())
    
except Exception as e:
    print(f"❌ LỖI KHI LƯU FILE: {e}")